In [1]:
import time
import multiprocessing

In [4]:
def count_down(x):
    while True:
        if x == 0:
            break
            
        print("카운트 다운: %d" % x)
        x -= 1
        time.sleep(1)
        
# 새로운 프로세스를 만들어서 count_down 함수를 실행시킴
p = multiprocessing.Process(target=count_down, args=(5, ))
p.start()

카운트 다운: 5
카운트 다운: 4
카운트 다운: 3
카운트 다운: 2
카운트 다운: 1


In [5]:
# ps -ef 를 통해 구동중인 프로세스를 확인할 수 있음
# cat /proc/cpuinfo 를 통해 CPU 개수를 확인할 수 있음
# CPU의 개수가 12개인데 프로세스(태스크)의 개수는 100개를 넘어감
# 이것이 가능한 이유는 무엇일까 ?
# 시분할 방식으로 동작하기 때문

# 프로세스란 ?
# 현재 실행중인 프로그램

# 우리는 C언어에서 기계어를 분석했음
# 컴퓨터에서 프로그램 코드가 동작하려면 반드시 뭘로 돌아가야 하는가 ? 기계어
# 기계어를 실행시킬 수 있는 녀석은 ? CPU 뿐
# 프로세스는 CPU의 추상화다.
# 프로세스는 CPU를 얻기 위해 끊임없이 경쟁한다.

# 기계어 코드가 어디 있나요 ? 실행 파일
# 운영체제에서 실행파일을 동작시킬 때 어디로 가는가 ? 메모리(물리 메모리)
# 실제 우리가 기계어 분석에서 봤던 메모리는 사실 가상 메모리
# 가상 메모리 -> 물리 메모리 변환 과정을 Pagination(페이지네이션)
# 기계어 코드는 어디에 배치되는가 ? TEXT 영역
# 프로그램을 실행하기 위해 먼저 TEXT를 배치함

In [9]:
def withdraw(money):
    for _ in range(20000):
        money.value -= 1
        
def deposit(money):
    for _ in range(20000):
        money.value += 1
        
def perform_transactions():
    money = multiprocessing.Value('i', 20000)
    
    proc1 = multiprocessing.Process(target=withdraw, args=(money, ))
    proc2 = multiprocessing.Process(target=deposit, args=(money, ))
    
    proc1.start()
    proc2.start()
    
    proc1.join()
    proc2.join()
    
    print("최종: {0}".format(money.value))
    
for _ in range(10):
    perform_transactions()

최종: 17306
최종: 18757
최종: 18456
최종: 20720
최종: 18247
최종: 23507
최종: 21737
최종: 18977
최종: 15560
최종: 19774


In [10]:
def withdraw(money, lock):
    for _ in range(20000):
        lock.acquire()
        money.value -= 1
        lock.release()
        
def deposit(money, lock):
    for _ in range(20000):
        lock.acquire()
        money.value += 1
        lock.release()
        
def perform_transactions():
    # 프로세스간 메모리 공유가 불가하므로
    # IPC(Inter Process Communication) 방식을 통해 정보를 공유하는 방식임
    money = multiprocessing.Value('i', 20000)
    lock = multiprocessing.Lock()
    
    proc1 = multiprocessing.Process(target=withdraw, args=(money, lock))
    proc2 = multiprocessing.Process(target=deposit, args=(money, lock))
    
    proc1.start()
    proc2.start()
    
    proc1.join()
    proc2.join()
    
    print("최종: {0}".format(money.value))
    
for _ in range(10):
    perform_transactions()

최종: 20000
최종: 20000
최종: 20000
최종: 20000
최종: 20000
최종: 20000
최종: 20000
최종: 20000
최종: 20000
최종: 20000


In [2]:
import threading

In [3]:
x = 0

def increment_glob():
    global x
    x += 1
    
def task_of_thread():
    for _ in range(10000000):
        increment_glob()
    
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = task_of_thread)
    t2 = threading.Thread(target = task_of_thread)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("x = {0}, iteration = {1}".format(x, i))

x = 18015756, iteration = 0
x = 17167138, iteration = 1
x = 17759298, iteration = 2
x = 17227001, iteration = 3
x = 17768409, iteration = 4
x = 17108170, iteration = 5
x = 16849072, iteration = 6
x = 17530167, iteration = 7
x = 17944958, iteration = 8
x = 17634908, iteration = 9


In [19]:
x = 0

lock = threading.Lock()

def increment_glob():
    global x
    x += 1
    
def task_of_thread():
    for _ in range(1000000):
        lock.acquire()
        increment_glob()
        lock.release()
    
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = task_of_thread)
    t2 = threading.Thread(target = task_of_thread)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("x = {0}, iteration = {1}".format(x, i))

Exception in thread Thread-124:
Traceback (most recent call last):
  File "/home/oem/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/oem/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-a868051f1164>", line 13, in task_of_thread
    lock.release()
RuntimeError: release unlocked lock



x = 2000001, iteration = 0
x = 2000000, iteration = 1
x = 2000000, iteration = 2
x = 2000000, iteration = 3
x = 2000000, iteration = 4
x = 2000000, iteration = 5
x = 2000000, iteration = 6
x = 2000000, iteration = 7
x = 2000000, iteration = 8
x = 2000000, iteration = 9


In [17]:
x = 0

lock = threading.Lock()

def increment_glob():
    global x
    x += 1
    
def task_of_thread():
    lock.acquire()
    
    for _ in range(1000000):
        increment_glob()
        
    lock.release()
    
def thread_main():
    global x
    x = 0
    
    # 스레드 구성
    t1 = threading.Thread(target = task_of_thread)
    t2 = threading.Thread(target = task_of_thread)
    
    # 스레드 시작
    t1.start()
    t2.start()
    
    # 스레드 종료까지 대기
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("x = {0}, iteration = {1}".format(x, i))

x = 2000000, iteration = 0
x = 2000000, iteration = 1
x = 2000000, iteration = 2
x = 2000000, iteration = 3
x = 2000000, iteration = 4
x = 2000000, iteration = 5
x = 2000000, iteration = 6
x = 2000000, iteration = 7
x = 2000000, iteration = 8
x = 2000000, iteration = 9
